Follow along this tutorial: https://github.com/alexeygrigorev/rag-agents-workshop

In [ ]:
!pip install minsearch

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [4]:
question = 'Can I still join the course?'

In [5]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [6]:
search_results = search(question)

In [7]:
prompt = build_prompt(question, search_results)

In [8]:
from dotenv import load_dotenv
load_dotenv()  # 會讀取當前目錄下 .env

True

In [9]:
!pip install -q -U google-genai

In [10]:
import requests
import json
import os
from google import genai

def llm(prompt):
  client = genai.Client()

  response = client.models.generate_content(
      model="gemini-2.5-flash", contents=prompt
  )
  return response.text

In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [12]:
rag("How do I patch KDE under FreeBSD?")


"I'm sorry, but the provided FAQ context does not contain any information about how to patch KDE under FreeBSD."

## "Agentic" RAG

In [13]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [14]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [15]:
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}


In [16]:
answer_json = llm(prompt)

In [17]:
print(answer_json)

{
"action": "SEARCH",
"reasoning": "The student is asking about course enrollment, specifically if they can still join. This is a common question related to course administration and enrollment deadlines, which is very likely to be addressed in an FAQ database."
}


In [18]:
import json

In [19]:
answer = json.loads(answer_json)

In [20]:
answer['action']

'SEARCH'

In [21]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [22]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [23]:
answer_json = llm(prompt)

In [24]:
print(answer_json)

```json
{
"action": "ANSWER",
"answer": "Yes, you can still join the course. Even if you haven't registered, you are eligible to submit the homeworks. You can just start learning and submitting homework without formal registration, as it's not checked against any registered list. However, be mindful of deadlines for final projects.",
"source": "CONTEXT"
}
```


## Agentic Search

In [25]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [44]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [45]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []

In [58]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [59]:
answer_json = llm(prompt)

In [60]:
answer = json.loads(answer_json)

In [61]:
previous_actions.append(answer)

In [62]:
keywords = answer['keywords']

In [63]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [64]:
search_results = dedup(search_results)

In [66]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [67]:
answer_json = llm(prompt)

In [69]:
print(answer['answer'])


KeyError: 'answer'

In [70]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [71]:
answer

{'action': 'ANSWER',
 'answer': 'To be successful in Module 1, which focuses on Docker and Terraform, based on the provided FAQ, you should be aware of and prepared to troubleshoot common technical issues related to setting up your environment. The FAQ highlights several specific problems and their solutions:\n\n1.  **Postgres/psycopg2 issues**: You might encounter `ModuleNotFoundError: No module named \'psycopg2\'` when working with Postgres. The solution often involves installing or upgrading `psycopg2-binary` (e.g., `pip install psycopg2-binary` or `pip install psycopg2-binary --upgrade`). If using Conda, updating it and reinstalling might be necessary. On a Mac, `brew install postgresql` could be required if `pg_config` is not found.\n2.  **SQLAlchemy errors**: When connecting to a PostgreSQL database using SQLAlchemy, you might face a `TypeError: \'module\' object is not callable` if your connection string is not correctly formatted. Ensure you use the `postgresql+psycopg://` dial

In [72]:
iteration

3

## Function calling ("tool use")

In [73]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [74]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [75]:
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [78]:
!pip install google-generativeai


INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 15.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [google-generativeai]ogle-ai-generativelanguage]


In [80]:
import google.generativeai as genai
from google.generativeai.types import Tool

# 🔑 設定你的 Gemini API 金鑰
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [76]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

NameError: name 'client' is not defined

In [2]:
import google.generativeai as genai
from google.generativeai.types import FunctionResponse

import json

# ✅ 初始化 Gemini
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ 定義 Search Tool（Function Calling 規格）

search_tool = [
    {
        "function_declarations": [
            {
                "name": "search",
                "description": "Search the FAQ database",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Search query text to look up in the course FAQ."
                        }
                    },
                    "required": ["query"]
                }
            }
        ]
    }
]



# ✅ 實作你自己的 search 函式（向量搜尋）
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )
    return results

# ✅ 包裝成 Gemini 可用的 function_call_output 回傳格式
def do_call(tool_call):
    function_name = tool_call.name
    arguments = dict(tool_call.args)  # ✅ 直接轉為 Python dict

    # Call 本地 function
    f = globals()[function_name]
    result = f(**arguments)

    # ✅ 使用 FunctionResponse 格式包裝結果回傳給 Gemini
    return FunctionResponse(
        name=function_name,
        response=json.dumps(result)  # 必須為 JSON string
    )

# ✅ 啟動聊天模型，支援 tools
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    tools=search_tool
)


# ✅ 使用者輸入
question = "How do I do well in module 1?"

chat = model.start_chat(history=[
    {
        "role": "user",
        "parts": [
            "You're a course teaching assistant. "
            "You're given a question from a course student and your task is to answer it. "
            "If you need to look up something in the FAQ, convert the student question into one or more queries."
        ]
    }
])

# ✅ 第一步：請 Gemini 判斷是否要用 Tool
response = chat.send_message(question)

# ✅ 如果 Gemini 要呼叫 Tool（function call），處理它
if response.candidates[0].content.parts and hasattr(response.candidates[0].content.parts[0], 'function_call'):
    tool_call = response.candidates[0].content.parts[0].function_call
    tool_result = do_call(tool_call)

    # ✅ 回傳 tool result 給 Gemini 讓它完成任務
    response = chat.send_message(tool_result)

# ✅ 最終回答
print("🤖 Gemini 回答：\n", response.text)


/home/horus/anaconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'FunctionResponse' from 'google.generativeai.types' (/home/horus/anaconda3/envs/llm/lib/python3.9/site-packages/google/generativeai/types/__init__.py)

In [98]:
!pip show  google-genai

Name: google-genai
Version: 1.25.0
Summary: GenAI Python SDK
Home-page: https://github.com/googleapis/python-genai
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: Apache-2.0
Location: /home/horus/anaconda3/envs/llm/lib/python3.9/site-packages
Requires: anyio, google-auth, httpx, pydantic, requests, tenacity, typing-extensions, websockets
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [87]:
calls = response.output

NameError: name 'response' is not defined

In [ ]:
for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [ ]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseOutputMessage(id='msg_686401ebe13c81918f6f3c84a1e7cec00e0676c3b7e4712d', content=[ResponseOutputText(annotations=[], text='To do well in Module 1, here are some tips based on common challenges and solutions:\n\n1. **Understand Your Environment**:\n   - Ensure that you have set up your development environment correctly. This includes installing Docker and Terraform as indicated in the module resources.\n\n2. **Common Installation Issues**:\n   - If you encounter errors such as `ModuleNotFoundError: No module named \'psycopg2\'`, try installing it via:\n     ```bash\n     pip install psycopg2-binary\n     ```\n   - If you\'re still encountering issues, consider updating pip or conda:\n     ```bash\n     pip install --upgrade pip\n     ```\n     or\n     ```bash\n     conda update -n base -c defaults conda\n     ```\n\n3. **PostgreSQL Connectivity**:\n   - When connecting to PostgreSQL using SQLAlchemy, ensure your connection string is formatted correctly, for example:\n     ```p

In [ ]:
for entry in response.output:
    chat_messages.append(entry)
    print(entry.type)

    if entry.type == 'function_call':      
        result = do_call(entry)
        chat_messages.append(result)
    elif entry.type == 'message':
        print(entry.text) 

In [ ]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [ ]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

 How do I do well in module 1?


function_call: ResponseFunctionToolCall(arguments='{"query":"module 1 tips"}', call_id='call_IED9lyUZOsS6ToxvZHLNQoN6', name='search', type='function_call', id='fc_686403087180819ebf792db8930e3f830594799b59703b57', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"query":"module 1 successful strategies"}', call_id='call_WAvKTixmfYyoBrF89lS6N46R', name='search', type='function_call', id='fc_68640309279c819ea24ded9b9639c7330594799b59703b57', status='completed')

To excel in Module 1, which focuses on Docker and Terraform, here are some key strategies and tips based on common challenges faced by students:

1. **Understand Docker Basics**: Start by familiarizing yourself with the Docker ecosystem, including images, containers, and Docker Compose. Having a solid grasp of these concepts will help you avoid common pitfalls.

2. **Environment Setup**: Ensure you have a correctly set up local environment. This includes installing Docker and any necessary dependencies. Fo

 Docker and Terraform


function_call: ResponseFunctionToolCall(arguments='{"query":"Docker tips for Module 1"}', call_id='call_zcxJlRqgjfMjavLOgVY1FSj5', name='search', type='function_call', id='fc_6864031f32a0819e850cbe6692b6ab160594799b59703b57', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"query":"Terraform tips for Module 1"}', call_id='call_4n2UHkf4UtPpg3XAM0V5WFfI', name='search', type='function_call', id='fc_6864031f61ec819eba1a01625d5a64430594799b59703b57', status='completed')

To do well in Module 1 focusing on Docker and Terraform, here are some targeted tips for both technologies:

### Docker Tips:
1. **Basic Understanding**: Ensure you have foundational knowledge of Docker, such as how to create and manage containers, and how to use Docker Compose for multi-container applications.

2. **Correct Installation**: If you encounter issues like `ModuleNotFoundError: No module named 'psycopg2'`, make sure that you're installing the required Python modules in your Docker envi

 stop


## Multiple tools

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-01 17:50:28--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3.4K) [text/plain]
Saving to: 'chat_assistant.py'

     0K ...                                                   100%  434K=0.008s

2025-07-01 17:50:28 (434 KB/s) - 'chat_assistant.py' saved [3485/3485]



In [ ]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [ ]:
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [ ]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(search, search_tool)

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}}]

In [ ]:
tools.add_tool(add_entry, add_entry_description)

In [ ]:
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add an entry to the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'The question to be added to the FAQ database'},
    'answer': {'type': 'string', 'description': 'The answer to the question'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [ ]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [ ]:
chat.run()

You: How do I do well in module 1?


You: add this to the FAQ database


You: stop


Chat ended.


In [ ]:
index.docs[-1]

{'question': 'How do I do well in module 1?',
 'text': '1. Understand the Basics: Ensure you grasp foundational concepts like Docker and Terraform.\n2. Practice Regularly: Engage with practical assignments to reinforce concepts.\n3. Utilize Resources: Refer to course materials, documentation, and suggested readings.\n4. Ask Questions: Reach out if you encounter difficulties; engaging with peers or instructors can clarify your understanding.\n5. Review Feedback: Take time to review feedback after completing tasks to guide improvements.\n6. Stay Organized: Keep your work organized to easily trace back errors.',
 'section': 'user added',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
index